In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import time
from gurobipy import *
import string


In [23]:
def preprocess(f):
    array = []
    for line in f:
        array.append(line)
    NameStr = array[0]
    tokens = NameStr.split()
    DataName = tokens[2]
    ComntStr = array[1]
    tokens = ComntStr.split()
    for i in range(0, len(tokens)):
        if tokens[i] == 'trucks:':
            break
    mt = tokens[i+1]
    min_trucks = ''.join([n for n in mt if n.isdigit()])
    min_trucks = int(min_trucks)
    
    for i in range(0, len(tokens)):
        if tokens[i] == 'value:':
            break
    opt = tokens[i+1]
    Optimal_val = ''.join([n for n in opt if n.isdigit()])
    Optimal_val = int(Optimal_val)
    
    dimstr = array[3]
    tokens = dimstr.split()
    Dim = tokens[2]
    Dim = int(Dim)
    
    capstr = array[5]
    tokens = capstr.split()
    Capacity = tokens[2]
    Capacity = int(Capacity)
    
    vertexdict = {}
    vertexnumber = []
    xc = []
    yc = []
    Vertices = []
    Arcs = []
    Cost={}
    i = 0
    for it in range(7, 7+Dim):
        p = array[it]
        tokens = p.split()
        vertexnumber.append(float(tokens[0]))
        xc.append(float(tokens[1]))
        yc.append(float(tokens[2]))
        Vertices.append((float(tokens[1]),float(tokens[2])))
    for vertex1 in Vertices:
        for vertex2 in Vertices:
            if vertex1 != vertex2:
                Arcs.append((vertex1,vertex2))
    
    for arc in Arcs:
        Cost[arc] = pow((pow(arc[0][0]-arc[1][0], 2) + pow(arc[0][1]-arc[1][1], 2)), 0.5)
        
        
    Demand={}
    Dem_sec = 7+Dim+1
    j = Dem_sec
    for vertex in Vertices:
        p = array[j]
        tokens = p.split()
        Demand[vertex] = int(tokens[1])
        j = j+1

    j = j+1
    p = int(array[j])
    depotx = xc[p-1]
    depoty = yc[p-1]
    depotnode = Vertices[p-1]
    return NameStr, Dim, Capacity, xc, yc, Vertices, Arcs, Cost, Demand, depotnode

In [44]:
def Printsolution(m, f, x, routes, A):
    if m.status == GRB.status.OPTIMAL:
        print('The data file is, %s' %f)
        print('The total cost is, %g' %m.objval)
        print('The edges used in the routes are:')
        for k in routes:
            for (i,j) in A:
                if(x[(i,j), k].x > 0.5):
                    print((i,j), end = " ; ")
            print('\n')
        print('The total time is, %g' %m.Runtime)
    elif m.Runtime >= m.Params.timelimit:
        print('The data file is, %s' %f)
        print('The total cost is, %g' %m.objval)
        print('The edges used in the routes are:')
        for k in routes:
            for (i,j) in A:
                if(x[(i,j), k].x > 0.5):
                    print((i,j), end = " ; ")
            print('\n')
        print('The total time is, %g' %m.Runtime)        
        
        

In [25]:
def Findneighbor(Graph, V, A):
    Neighbors = {}
    for vertex in V:
        Neighbors[vertex] = []
        for arc in A:
            if(arc[0] == vertex):
                if(arc[1] not in Neighbors[vertex]):
                    Neighbors[vertex].append(arc[1])
            elif(arc[1] == vertex):
                if(arc[0] not in Neighbors[vertex]):
                    Neighbors[vertex].append(arc[0])
    return Neighbors  

def Findcomponents(Graph, V, A):
    Components = []
    visited = {}
    for v in V:
        visited[v] = 0
    for v in V:
        if(visited[v] == 0):
            component = []
            dfs(Graph, V, A, v, component, visited)
            Components.append(component)
    return Components

def dfs(Graph, V, A, v, component, visited):
    visited[v] = 1
    component.append(v)
    neighbs = Findneighbor(Graph, V, A)
    for v2 in neighbs[v]:
        if(visited[v2] == 0):
            dfs(Graph, V, A, v2, component, visited)
            
    

In [53]:
def solveCVRP(f):
    file, dim, cap, x, y, V, A, C, demand, depot = preprocess(f)
    
    VRP = Model("CVRP_General_Subtour_Lazy")
    x = {}
    routes = list(range(1, 11))
    for k in routes:
        for (i,j) in A:
            x[(i, j), k] = VRP.addVar(vtype=GRB.BINARY, name='x_%i_%j_%k')

    # Update model to integrate new variables
    VRP.update()

    # The objective is to minimize distances of the path
    VRP.setObjective(quicksum(x[(i,j), k] * C[(i,j)] for (i,j) in A for k in routes), GRB.MINIMIZE)

    # Constraints
    for i in V:
        if i != depot:
            # every single vertex is visited exactly once (except depot)
            VRP.addConstr(quicksum( x[(i,j), k] for j in V if j !=i for k in routes) == 1, name ='cons1' )
            # vehicle visit and leave the vertex
            for k in routes:
                VRP.addConstr(quicksum(x[(i,j), k] for j in V if j!=i) - quicksum(x[(j,i), k] for j in V if j!=i) == 0 , name ='cons2' )

    for k in routes:
        # vehicle leaves and comes back to the depot
        VRP.addConstr(quicksum( x[(depot, i), k] for i in V if i != depot) <= 1, name ='cons3' )
        VRP.addConstr(quicksum( x[(i, depot), k] for i in V if i != depot ) <= 1, name ='cons4' )
        # capacity of the route
        VRP.addConstr(quicksum(demand[i] * x[(i,j), k] for i in V for j in V if j !=i ) <= cap, name ='cons5' )
    
    VRP._x = x
    # set run timelimit:
    VRP.Params.timelimit = 16
    VRP.Params.lazyConstraints = 1
    def subtour_cut(VRP, where):
        if where == GRB.Callback.MIPSOL:
            sol = VRP.cbGetSolution(VRP._x)
            arcs_used = []
            for arc in sol:
                if (sol[arc] >= 0.9):
                    if(arc[0][0] != depot or arc[0][1] != depot):
                        arcs_used.append(arc[0])
            A = arcs_used
            Newgraph = (V, A)
            
            Components = Findcomponents(Newgraph, V, A)
            if(len(Components) > 1):
                #for com in Components:
                #    print(com)
                #    print('\n')
                for k in routes:
                    for component in Components:
                        VRP.cbLazy( quicksum(x[(i,j), k] for i in component if i != depot for j in component if j!=i if j!= depot ) <= len(component) - 1)
                        
    VRP.optimize(subtour_cut)
    dep = str(depot)
    print(" ##########################################################################################")
    print("The depot is %s" %dep)
    Printsolution(VRP, file, x, routes, A)

In [54]:
f = open("C:\\Users\\Prasangsha\\Documents\\A-n5.vrp", 'r')
#file, dim, cap, x, y, V, A, C, demand, depot = preprocess(f)

In [35]:
C

{((82.0, 76.0), (96.0, 44.0)): 34.92849839314596,
 ((82.0, 76.0), (50.0, 5.0)): 77.87810988975015,
 ((82.0, 76.0), (49.0, 8.0)): 75.58438992278762,
 ((82.0, 76.0), (13.0, 7.0)): 97.58073580374356,
 ((96.0, 44.0), (82.0, 76.0)): 34.92849839314596,
 ((96.0, 44.0), (50.0, 5.0)): 60.30754513325841,
 ((96.0, 44.0), (49.0, 8.0)): 59.20304046246274,
 ((96.0, 44.0), (13.0, 7.0)): 90.87353850269065,
 ((50.0, 5.0), (82.0, 76.0)): 77.87810988975015,
 ((50.0, 5.0), (96.0, 44.0)): 60.30754513325841,
 ((50.0, 5.0), (49.0, 8.0)): 3.1622776601683795,
 ((50.0, 5.0), (13.0, 7.0)): 37.05401462729781,
 ((49.0, 8.0), (82.0, 76.0)): 75.58438992278762,
 ((49.0, 8.0), (96.0, 44.0)): 59.20304046246274,
 ((49.0, 8.0), (50.0, 5.0)): 3.1622776601683795,
 ((49.0, 8.0), (13.0, 7.0)): 36.013886210738214,
 ((13.0, 7.0), (82.0, 76.0)): 97.58073580374356,
 ((13.0, 7.0), (96.0, 44.0)): 90.87353850269065,
 ((13.0, 7.0), (50.0, 5.0)): 37.05401462729781,
 ((13.0, 7.0), (49.0, 8.0)): 36.013886210738214}

In [55]:
solveCVRP(f)

Changed value of parameter timelimit to 16.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 74 rows, 200 columns and 720 nonzeros
Model fingerprint: 0xa3c9295e
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 571.9434680
Presolve removed 0 rows and 60 columns
Presolve time: 0.01s
Presolved: 74 rows, 140 columns, 480 nonzeros
Variable types: 0 continuous, 140 integer (140 binary)

Root relaxation: objective 2.713430e+02, 64 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  271.34302    0  